In [ ]:
!pip install ipdb

In [22]:
import os
from ollama import Client
import requests


client = Client(
  host="http://localhost:7869",
  # auth_token="sk-bae3d6cde76e4ebda945748a706d64f1"
)

DEFAULT_MODEL = "deepseek-r1:70b"

os.environ["OPENWEATHER_API_KEY"] = "1e15dc16facdc4f0b91b2bb3fac53000"
def get_weather(city: str) -> str:
    """Get the weather for a city"""
    latitude, longitude = get_coordinates(city)
    weather = get_weather_by_coordinates(latitude, longitude)
    return f"The weather in {city} is {weather}."



def get_weather_by_coordinates(latitude: float, longitude: float) -> str:
    """Get the weather for a latitude and longitude"""
    response = requests.get(f"https://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={os.getenv('OPENWEATHER_API_KEY')}")
    data = response.json()

    if response.status_code != 200:
        raise ValueError(f"Error getting weather: {data.get('message', 'Unknown error')}")

    # Extract weather description from the API response
    weather_description = data['weather'][0]['description']
    temperature_kelvin = data['main']['temp']
    temperature_celsius = temperature_kelvin - 273.15  # Convert Kelvin to Celsius

    return f"The weather at {latitude}, {longitude} is {weather_description} with temperature {temperature_celsius:.1f}°C"


def get_coordinates(city: str) -> tuple[float, float]:
    """Get the latitude and longitude for a city name"""
    geocoding_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={os.getenv('OPENWEATHER_API_KEY')}"
    response = requests.get(geocoding_url)
    data = response.json()

    if not data:
        raise ValueError(f"City '{city}' not found")

    location = data[0]
    return location['lat'], location['lon']

# # show ["Shanghai", "Beijing", "Zhengzhou"] weather
# for city in ["Shanghai", "Beijing", "Zhengzhou"]:
#     print(get_weather(city))

# # show ["Shanghai", "Beijing", "Zhengzhou"] coordinates
# for city in ["Shanghai", "Beijing", "Zhengzhou"]:
#     print(get_coordinates(city))

def get_tools_description():
    """Return the list of available tools/functions with their descriptions"""
    return [
        {
            "name": "get_weather",
            "description": "Get the current weather for a specified city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get weather for"
                    }
                },
                "required": ["city"]
            }
        },
        {
            "name": "get_coordinates",
            "description": "Get the latitude and longitude coordinates for a specified city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get coordinates for"
                    }
                },
                "required": ["city"]
            }
        },
        {
            "name": "get_weather_by_coordinates",
            "description": "Get the current weather for specified latitude and longitude",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "number",
                        "description": "The latitude coordinate"
                    },
                    "longitude": {
                        "type": "number",
                        "description": "The longitude coordinate"
                    }
                },
                "required": ["latitude", "longitude"]
            }
        }
    ]

def execute_tool(tool_name: str, parameters: dict) -> str:
    """Execute a tool with the given parameters"""
    available_tools = {
        "get_weather": get_weather,
        "get_coordinates": get_coordinates,
        "get_weather_by_coordinates": get_weather_by_coordinates
    }

    if tool_name not in available_tools:
        raise ValueError(f"Unknown tool: {tool_name}")

    tool = available_tools[tool_name]
    return tool(**parameters)


In [27]:
def chat_with_tools(messages: list, tools: list = None) -> str:
    """
    Chat with the LLM using tools, similar to OpenAI's function calling system
    """
    if tools is None:
        tools = get_tools_description()

    # Format the system message to include tools information
    system_message = {
        "role": "system",
        "content": f"""You are a helpful assistant with access to the following tools:
{tools}

When you need to use a tool, respond with a JSON object in this format:
{{
    "tool": "tool_name",
    "parameters": {{
        "param1": "value1",
        "param2": "value2"
    }}
}}

Only respond with the JSON object when you need to use a tool."""
    }

    # Add system message at the beginning
    full_messages = [system_message] + messages

    # Get response from Ollama
    response = client.chat(model=DEFAULT_MODEL, messages=full_messages)

    try:
        # Try to parse the response as a tool call
        import json
        tool_call = json.loads(response['message']['content'])

        if 'tool' in tool_call and 'parameters' in tool_call:
            # Execute the tool
            result = execute_tool(tool_call['tool'], tool_call['parameters'])

            # Add the tool result to the conversation
            full_messages.append({
                "role": "assistant",
                "content": response['message']['content']
            })
            full_messages.append({
                "role": "system",
                "content": f"Tool response: {result}"
            })

            # Get final response from Ollama
            final_response = client.chat(model=DEFAULT_MODEL, messages=full_messages)
            return final_response['message']['content']

    except (json.JSONDecodeError, KeyError):
        # If the response is not a tool call, return it directly
        return response['message']['content']

In [28]:
messages = [{
        "role": "user",
        "content": "What's the weather like in Beijing?"
    }]

response = chat_with_tools(messages)
print("Response:", response)

Response: <think>
Okay, so the user is asking about the weather in Beijing. I have three tools at my disposal: get_weather, which requires a city name; get_coordinates, also requiring a city; and get_weather_by_coordinates, which needs latitude and longitude.

Since the first tool, get_weather, directly takes a city name, it's the most straightforward to use here. The user provided "Beijing," so I can immediately call this tool without needing any intermediate steps like finding coordinates first.

I don't think there's any ambiguity in the request. The user just wants the current weather conditions for Beijing. Using get_weather is efficient because it directly fetches the necessary information without requiring additional lookups or steps that might slow down the process.

So, I'll structure a JSON response using the get_weather tool with "Beijing" as the parameter.
</think>

{
    "tool": "get_weather",
    "parameters": {
        "city": "Beijing"
    }
}
